# Fundamental Factor Models

Fundamental factor models start with asset returns $r_t$ and factor loadings $B_t$, and estimates factor returns $f_t$ and idio returns $\epsilon_t$

Process:
- Data ingestion: correctness, outliers, consistency across vendors, missing data
- Estimation universe selection
- Winsorization: identify outliers and winsorize
- Loading generation: generate $B_t$
- Cross-sectional regression: estimate $f_t$ and $\epsilon_t$
- Time-series estimation:
  - factor cov matrix
  - idio cov matrix
  - risk-adjusted performance of factor returns

## Cross-sectional regression

Starting with a single period model:

$$ r_t = Bf_t + \epsilon_t $$

Where $r_t$ and $f_t$ are column vectors. To estimate $f_t$, we solve the following optimization problem:

$$ \min ||r_t - Bf_t||^2 $$
$$ \text{s.t. } f \in R^{m}$$

However, since the idio returns $\epsilon_t$ usually aren't homoskedastic (same variance), we need to make them so:

$$ \Omega_{\epsilon}^{-1/2}r_t = \Omega_{\epsilon}^{-1/2}Bf_t + \Omega_{\epsilon}^{-1/2}\epsilon_t $$
$$ var(\Omega_{\epsilon}^{-1/2}\epsilon_t) = 
    \Omega_{\epsilon}^{-1/2}\Omega_{\epsilon}\Omega_{\epsilon}^{-1/2} = I_m
$$

And the optimization problem becomes

$$ \min ||\Omega_{\epsilon}^{-1/2}(r_t - Bf_t)||^2 $$
$$ \text{s.t. } f \in R^{m}$$

The solution is the normal equation: 

$$ \hat{f}_t = (B^T\Omega_{\epsilon}^{-1}B)^{-1}B^T\Omega_{\epsilon}^{-1}r_t $$

For multi-period case where the factor returns $F \in R^{mxT}$, the method stays the same as each single-period factor returns can be solved independently and combined to get the factor returns matrix $F$

### Idio covariance matrix

We don't know idio cov matrix before solving for factor returns, and we need idio cov matrix to solve for factor returns. It's a chicken and egg problem. We can borrow from Expectation-Maximization procedure, 
- we start with an identity idio cov matrix
- solve for factor returns and get a new idio returns and cov matrix
- solve for factor returns again using the new idio returns, get new factor returns and update the idio cov matrix.

If we have $T$ time stamps, we can use the first half of the samples to estimate idio cov matrix, then use and update it in a walk-forward manner
- Use the idio cov matrix from time $t-1$ to estimate factor returns and idio returns at time $t$
- Update idio cov matrix with the new idio returns from time $t$ 

### Rank-deficient Loading Matrices

An example would be, a country factor and an industry factor, each asset must belong to one and only one country and one industry. Therefore, the sum of all country loadings equals the sum of all industry loadings equals a vector of ones. A vector that assign 1 to all country loadings and -1 to all industry loadings and 0 everywhere else will create a 0 vector, thus the loading matrix is rank-deficient due to non-empty null space. If $B$ is rank-deficient, so is $B^TB$ and we can't inverse it, therefore we can't get the factor returns estimate $\hat{f}_t$.

Ways to deal with this:
- Add ridge regularization: $||r-Bf||^2 + \delta ||f||^2$. As $\delta \to 0$, this becomes ridgeless, and we can calculate the peudo-inverse of $B^TB$ for factor returns estimate.
- Or, we can add constraint, for example: $w^TB_{ind}f_{ind}=0$, where $w$ is the firm's market cap weight, which indicates that the market-weighted industry returns must be 0.

## Estimating Factor Covariance Matrix

$$\Omega_T^{emp} = T^{-1}\sum^T_{t=1}\hat{f}_t\hat{f}_t^T$$
The standard error is $\sqrt{2/T}$, so as $T \to \infty$, $\Omega_T^{emp} \to \Omega_f$

The issues are:
- The number of factor is not much smaller than the number of observations, then the estimate is noisy
- Factor return estimate is inflated by the estimation process
- Factor returns are non-stationary during crisis
- Factor returns are mildy autocorrelated

### Factor covariance matrix shrinkage

The estimate $var(\hat{f}_t)$ is biased:

$$var(\hat{f}_t) = \Omega_f + (B^T\Omega_{\epsilon}^{-1}B)^{-1}$$

**WHY?????**

To correct this, we set
$$ \hat{\Omega}_f = var(\hat{f}_t) - (B^T\Omega_{\epsilon}^{-1}B)^{-1}$$

Combined with the Ledoit-Wolf shrinkage

$$ \Omega_{f, shrink}(\rho) = (1-\rho)\hat{\Omega}_f + \rho\frac{trace(\hat{\Omega}_f)}{m}I_m$$

### Dynamic Conditional Correlation

The idea is to estimate the correlation matrix and factor volatilities separately using EMA, where correlation matrix changes slowly while factor vols change fast:

$$\Omega_f = VCV$$
$$diag(V^2) = \kappa_V\sum^T_{s=0}e^{-s/\tau_V}\hat{f}_{t-s} \circ \hat{f}_{t-s}$$
$$C = \kappa_C\sum^T_{s=0}e^{-s/\tau_C}V_{t-s}^{-1}\hat{f}_{t-s}\hat{f}_{t-s}^TV_{t-s}^{-1}$$

Where $\tau_V < \tau_C$ and the $\kappa$ terms are normalizing terms.

### Short-Term Volatility Updating (STVU)

This can be applied in addition to the dyanmic conditional correlation when we have estimated $V$ and $C$. Which aims at further adjusting the covariance matrix for more flexibility to sudden increase and decrease in volatility.

$$f_t = e^{x_t/2} V_t C_t^{1/2} \eta_t$$
$$\eta_t \sim N(0, I_m)$$
$$x_{t+1} = \phi x_t + \sigma \gamma_t$$

If $x_t = 0$ for all t, then it's just the regular factor cov. Set:

$$u_t = C_t^{-1/2}V_t^{-1}f_t$$
$$x_t = log||u_t||^2 - log||\eta_t||^2$$

Where $u_t$ is the whittened factor returns and $x_t$ is derived from $f_t = e^{x_t/2} V_t C_t^{1/2} \eta_t$. Set:
$$y_t = x_t + \epsilon_t = x_t + \left(log||\eta_t||^2 - E(log||\eta_t||^2)\right)$$

Combined with:
$$x_{t+1} = \phi x_t + \sigma \gamma_t$$

It's a state-space model, and $x_t$ can be solved as:

$$x_t = (1 - K) \sum^{\infty}_{s=0} K^s \left(x_{t-s}\right)$$
$$x_t = (1 - K) \sum^{\infty}_{s=0} K^s \left(log||u_{t-s}||^2 - log||\eta_{t-s}||^2\right)$$
$$x_t = (1 - K) \sum^{\infty}_{s=0} K^s log||u_{t-s}||^2 - 
    (1 - K) \sum^{\infty}_{s=0} K^s log||\eta_{t-s}||^2$$
$$x_t = (1 - K) \sum^{\infty}_{s=0} K^s log||u_{t-s}||^2 - 
    E(log||\eta_t||^2)$$
$$x_t = (1 - K) \sum^{\infty}_{s=0} K^s \left(log||u_{t-s}||^2 - E(log||\eta_t||^2)\right)$$

Set $K = e^{-1/\tau_0}$ where $\tau$ is the half-life (tunable):

$$x_t = (1 - e^{-1/\tau_0}) \sum^{\infty}_{s=0} 
    e^{-s/\tau_0} \left(log||u_{t-s}||^2 - E(log||\eta_t||^2)\right)$$
$$e^{x_t/2} = exp\left[\frac{1 - e^{-1/\tau_0}}{2} \sum^{\infty}_{s=0} 
    e^{-s/\tau_0} \left(log||u_{t-s}||^2 - E(log||\eta_t||^2)\right)\right]$$

Simplify:

$$E(log||\eta_t||^2) = E(\log m ||\eta_t||^2 / m) = log(m) + E(log(||\eta||^2 / m) \simeq \log{m}$$

The last sim equal is because $\eta$ has unit variance, and low of large number makes  $||\eta_t||^2 / m \to 1$ as $m \to \infty$:

\begin{align}
e^{x_t/2} & \simeq \exp\left[\frac{1 - e^{-1/\tau_0}}{2} \sum^{\infty}_{s=0} 
    e^{-s/\tau_0} \left(\log||u_{t-s}||^2 - \log m\right)\right] \\ 
    & = \exp\left[(1 - e^{-1/\tau_0}) \sum^{\infty}_{s=0} 
    e^{-s/\tau_0} \left(\log(||u_{t-s}||^2/m)^{1/2}\right)\right] \\ 
    & = \exp\left[(1 - e^{-1/\tau_0}) \sum^{\infty}_{s=0} 
    e^{-s/\tau_0} \left(\log(||u_{t-s}||/\sqrt{m})\right)\right] \\ 
    & \simeq \exp\left[(1 - e^{-1/\tau_0}) \sum^{\infty}_{s=0} 
    e^{-s/\tau_0} \left(||u_{t-s}||/\sqrt{m} - 1\right)\right] \\ 
    & = \exp\left[(1 - e^{-1/\tau_0}) \left(\sum^{\infty}_{s=0} 
    e^{-s/\tau_0} ||u_{t-s}||/\sqrt{m}\right) - 1\right] \\ 
    & \simeq \exp\left[\log\left((1 - e^{-1/\tau_0}) \left(\sum^{\infty}_{s=0} 
    e^{-s/\tau_0} ||u_{t-s}||/\sqrt{m}\right)\right)\right] \\ 
    & = (1 - e^{-1/\tau_0}) \sum^{\infty}_{s=0} 
    e^{-s/\tau_0} ||u_{t-s}||/\sqrt{m}
\end{align}

The second and third sim equalities are linear approximation of the log operator: $log(1+x) \simeq x$ for some small $x$.

Since $u$ is the whittened factor returns, and $||u||/\sqrt(m)$ is the standard deviation:
- When this std is 1, no adjustment to the factor returns
- When this std is greater than 1, adjust factors upward
- When this std is less than 1, adjust factors downward

### Correcting for autocorrelation in Factor Returns

Factor returns are slightly autocorrelated in the short term, so we add in the autocovariance into the cov matrix estimate:

$$[C_l]_{i,j} = cov(f_{t,i}, f_{t-l, j})$$
$$\hat{\Omega}_f = C_0 + \frac{1}{2} \sum^{l_{max}}_{l=1}(C_l + C_l^T)$$

For a more advanced adjustment:

$$\hat{\Omega}_f = C_0 + \sum^{l_{max}}_{l=1}\left(1 - \frac{l}{1+l_{max}}\right)(C_l + C_l^T)$$

The intuition is: adjust the covariance matrix upward, if there is a positive autocorrelation, and vice versa.

**WHY???**

## Estimating Idio Cov Matrix

### Exponential Weighting

We have an error vector at each time, $\epsilon_t$. Let $E \in R^{n \times T}$ by the collections of the error vectors over time, e.g. each column is an error vector at a time. To estimate the covariance matrix $\hat{\Omega}_{\epsilon}$, we use an exponential moving average: 

$$\hat{\Omega}_{\epsilon} = EWE^T$$

Where the matrix $W$ is a positive definite diagonal matrix. For a simple averaging, $W=I_T$. But for exponential weighting, we set $[W]_{t,t} = \kappa \exp(-t/\tau)$ where $\tau$ is the half life and $\kappa$ is the normalizing term such that $trace(W) = T$

### Visual Inspection

This matrix should be diagonal (no correlation between assets and across time), or at least sparse. But the reality is usually neither, so it helps to check the matrix visually as there might be some leftover covariance that can be explained by additional factors. For example, a stock may have different share classes that are highly correlated but not identicial, so a share class factor would help remove such covariance in the idio covariance matrix

### Short-Term Idio Update

This is similar to STVU, but since the idio matrix should be diagonal, the update is simplified.

\begin{align}
e^{\hat{x}_t/2} & = \kappa_0 \sum^{\infty}_{s=0} 
    e^{-s/\tau_0} 
    \sqrt{
        \frac{\sum_i a_{i,t-s}(\epsilon_{i,t-s} / \hat{\sigma}_{i,t-s})^2}
            {\sum_i a_{i,t-s}}
        }\\
a_{i,t} &= \begin{cases}
    1 - |t-T_{earn,i}|/\tau_{earn}, \text{if } |t-T_{earn,i}| \leq \tau_{earn} \\
    0, \text{otherwise}
\end{cases}
\end{align}

$a_{i,t}$ is a tent-shaped variable that increases as we get close to the event and reduces back to 0 when we are outside of the event impact window $\tau_{earn}$. The volatility adjustment $\exp(\hat{x}_t/2)$ is quite similar to STVU where $\epsilon_{i,t}/\hat{\sigma}_{i,t}$ is the whitened noise (no error correlations). In addition to the exponential weighting from STVU, we add the $a_{i,t}$ adjustment for a weighted estimate of the idio variance at each timestamp, note that $a_{i,t}$ is assigned to each asset at each time stamp, earnings schedule is different for different firms. Since $a_{i,t}=0$ when the firm isn't close to any event, the overall adjuster $\exp(\hat{x}_t/2)$ is muted.

$$ \hat{\sigma}_{i,t}^2 \leftarrow \left[(1-a_{i,t}) + a_{i,t}e^{\hat{x}_t}\right] \hat{\sigma}_{i,t}^2$$

### Off-Diagonal Clustering

Some assets might be highly correlated because a) same firm different share classses b) similarity between firms, VISA and MasterCard, etc. We can use a correlation threshold to identify the clusterings

$$thres_{\lambda}(\rho_{i,j}) = \rho_{i,j}1\{|\rho_{i,j} > \lambda\}$$
$$\lambda = K\sqrt{\log {n/T}}$$

Basically, pick a positive constant, and the correlation threshold $\lambda$ should be higher if we have more assets or shorter time horizon. 

### Matrix Shrinkage

This is the same as factor cov matrix shrinkage, expcept that we don't need to deal with the autocovariance:

$$ \Omega_{\epsilon, shrink}(\rho) = (1-\rho)\hat{\Omega}_{\epsilon} + \rho\frac{trace(\hat{\Omega}_{\epsilon})}{m}I_m$$

We are just pulling the covariance matrix to the average of variances.

## Winsorization of Returns

Historical data can be wrong because: 1) data recording issue, wrong data, 2) truly outliers caused by real events. 
Use robust z-score to spot outliers:

$$ d_{i,t} = \frac{|\log (1+r_{i,t})|}{\text{median}(|\log(1+r_{i,t-1})|,\ldots,|\log(1+r_{i,t-T})|}$$

$T$ is the lookback window, and threshold is set somewhere between 5 and 10. Make sure to record, report and examine each of the outliers.
